In [ ]:
!pip install protobuf>=5.29.1
!pip install git+https://github.com/huggingface/parler-tts.git
!pip install flask flask_cors pyngrok soundfile scipy
!pip install vosk==0.3.45
!apt-get install -y libsndfile1
!wget https://alphacephei.com/vosk/models/vosk-model-small-hi-0.22.zip
!unzip vosk-model-small-hi-0.22.zip

  Cloning https://github.com/huggingface/parler-tts.git to /tmp/pip-req-build-1jaegk29
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/parler-tts.git /tmp/pip-req-build-1jaegk29
  Resolved https://github.com/huggingface/parler-tts.git to commit d108732cd57788ec86bc857d99a6cabd66663d68
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/descriptinc/audiotools to /tmp/pip-install-45_fioib/descript-audiotools_f63bf8d26862441287dce892102d8e42
  Running command git clone --filter=blob:none --quiet https://github.com/descriptinc/audiotools /tmp/pip-install-45_fioib/descript-audiotools_f63bf8d26862441287dce892102d8e42
  Resolved https://github.com/descriptinc/audiotools to commit 348ebf2034ce24e2a91a553e3171cb00c0c71678
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.0 MB/s eta 0:00:00
  Prep

In [ ]:
from flask import Flask, request, send_file, jsonify
from flask_cors import CORS
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import torch
import soundfile as sf
import scipy.io.wavfile
import numpy as np
from pyngrok import ngrok
import os
from vosk import Model as VoskModel, KaldiRecognizer
import wave
import json
import uuid

In [ ]:
app = Flask(__name__)
CORS(app)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [ ]:
print("Loading Parler-TTS model...")
tts_model = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(device)
prompt_tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
desc_tokenizer = AutoTokenizer.from_pretrained(tts_model.config.text_encoder._name_or_path)

Loading Parler-TTS model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.75G [00:00<?, ?B/s]

  "_name_or_path": "google/flan-t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "transformers_version": "4.46.1",
  "use_cache": true,
  "vocab_size": 32128
}

  "_name_or_path": "ylacombe/dac_44khz",
  "architectures": [
    "DacModel"
  ],
  "codebook_dim": 8,
  "codebook_loss_weight": 1.0,
  "codebook_size": 1024,
  "commitment_loss_weight": 0.25,
  "decoder_hidden_si

...

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
print("Loading Vosk model...")
vosk_model = VoskModel("vosk-model-small-hi-0.22")

Loading Vosk model...


In [ ]:
@app.route('/tts', methods=['POST'])
def tts():
    data = request.get_json()
    if not data or 'text' not in data:
        return jsonify({"error": "Missing 'text' parameter"}), 400

    prompt = data['text']
    description = "A female speaker delivers a slightly expressive and animated speech with a slow speed and pitch. The recording is of very high quality, with the speaker's voice sounding clear and very close up."

    try:
        desc_ids = desc_tokenizer(description, return_tensors="pt").to(device)
        prompt_ids = prompt_tokenizer(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            output = tts_model.generate(
                input_ids=desc_ids.input_ids,
                attention_mask=desc_ids.attention_mask,
                prompt_input_ids=prompt_ids.input_ids,
                prompt_attention_mask=prompt_ids.attention_mask
            )

        audio_arr = output.squeeze().cpu().numpy()
        output_path = f"/tmp/{uuid.uuid4()}.wav"
        sf.write(output_path, audio_arr, samplerate=tts_model.config.sampling_rate)

        return send_file(output_path, mimetype="audio/wav", as_attachment=True)

    except Exception as e:
        print("TTS Error:", str(e))
        return jsonify({"error": str(e)}), 500


In [ ]:
@app.route('/stt', methods=['POST'])
def stt():
    if 'audio' not in request.files:
        return jsonify({'error': 'Missing audio file'}), 400

    audio_file = request.files['audio']
    wf = wave.open(audio_file, 'rb')
    recognizer = KaldiRecognizer(vosk_model, wf.getframerate())
    recognizer.SetWords(True)

    result_text = []

    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if recognizer.AcceptWaveform(data):
            res = json.loads(recognizer.Result())
            result_text.append(res.get('text', ''))

    final_res = json.loads(recognizer.FinalResult())
    result_text.append(final_res.get('text', ''))

    return jsonify({'text': ' '.join(result_text)})

In [ ]:
ngrok.set_auth_token("2wPDgUvkvFjtmraTwQLUcQ6tDPv_4YQFExk8HcddoxTNLvehD")

In [ ]:
public_url = ngrok.connect(5001)
print(f"🔥 Ngrok tunnel running at: {public_url}")

# Run the app
if __name__ == '__main__':
    app.run(port=5001)

🔥 Ngrok tunnel running at: NgrokTunnel: "https://942f-34-87-3-101.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/May/2025 17:10:28] "POST /tts HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/May/2025 17:43:51] "POST /tts HTTP/1.1" 200 -
